In [1]:
import pandas as pd
import json

#------------CLEANING WEATHER FILE------------#

with open('weather_data.json', 'r') as file:
    data = json.load(file)

records = []
for city,instances in data.items():
    for city_info in instances:
        city_info = city_info['data']
        flat = pd.json_normalize(city_info, sep='_').to_dict(orient='records')[0]
        flat['city'] = city
        
        weather_info = pd.json_normalize(city_info['weather'], sep='_').to_dict(orient='records')[0]
        for key, value in weather_info.items():
            flat[f'weather_{key}'] = value
        flat.pop('weather', None)
        records.append(flat)

df_weather = pd.DataFrame(records)
# df_weather.set_index('city', inplace=True)
df_weather['dt'] = pd.to_datetime(df_weather['dt'], unit='s')
df_weather['sys_sunrise'] = pd.to_datetime(df_weather['sys_sunrise'], unit='s')
df_weather['sys_sunset'] = pd.to_datetime(df_weather['sys_sunset'], unit='s')

df_weather

,base,visibility,dt,timezone,id,name,cod,coord_lon,coord_lat,main_temp,...,sys_country,sys_sunrise,sys_sunset,city,weather_id,weather_main,weather_description,weather_icon,wind_gust,rain_1h
0,stations,10000,2025-08-21 00:20:17,10800,7922173,Al ‘Atabah,200,31.2357,30.0444,299.48,...,EG,2025-08-21 03:25:59,2025-08-21 16:30:48,cairo,800,Clear,clear sky,01n,NaN,NaN
1,stations,10000,2025-08-21 00:18:25,7200,756135,Warsaw,200,21.0122,52.2297,286.54,...,PL,2025-08-21 03:29:23,2025-08-21 17:49:11,warsaw,800,Clear,clear sky,01n,NaN,NaN
2,stations,10000,2025-08-21 00:20:18,28800,1816670,Beijing,200,116.3913,39.9062,299.09,...,CN,2025-08-20 21:31:47,2025-08-21 11:03:53,beijing,501,Rain,moderate rain,10d,2.91,2.73


In [2]:
#------------CLEANING Pollution FILE------------#

with open('pollution_data.json') as file:
    data = json.load(file)

pollution_records = []
for city, instances in data.items():
        for city_poll in instances:
            city_poll= city_poll['data']
            city_poll_flat = pd.json_normalize(city_poll, sep='_').to_dict(orient='records')[0]
            city_poll_flat['city'] = city

            pollution_info = pd.json_normalize(city_poll_flat['list'], sep="_").to_dict(orient='records')[0]
            for key, value in pollution_info.items():
                city_poll_flat[f'pollution_{key}'] = value
            city_poll_flat.pop('list', None)

            pollution_records.append(city_poll_flat)

df_pollution = pd.DataFrame(pollution_records)
# df_pollution.set_index('city', inplace=True)
df_pollution['pollution_dt'] = pd.to_datetime(df_pollution['pollution_dt'], unit='s')

df_pollution

,coord_lon,coord_lat,city,pollution_dt,pollution_main_aqi,pollution_components_co,pollution_components_no,pollution_components_no2,pollution_components_o3,pollution_components_so2,pollution_components_pm2_5,pollution_components_pm10,pollution_components_nh3
0,31.2357,30.0444,cairo,2025-08-21 00:20:17,2,125.13,0.00,2.35,95.45,7.64,15.13,43.57,2.20
1,21.0067,52.2320,warsaw,2025-08-21 00:20:18,1,143.78,0.00,4.29,59.71,1.51,2.48,3.66,2.86
2,116.3913,39.9062,beijing,2025-08-21 00:20:19,5,434.70,2.14,18.55,4.46,11.23,84.24,89.73,13.21


In [3]:
try:
    df_pollution.drop(columns=['coord_lon', 'coord_lat'], inplace=True)
except:
    pass
df_merged= df_weather.merge(
    df_pollution,
    on='city',
    how='inner'
)
df_merged.set_index('city', inplace=True)
df_merged.drop_duplicates(inplace=True)
df_merged


,base,visibility,dt,timezone,id,name,cod,coord_lon,coord_lat,main_temp,...,pollution_dt,pollution_main_aqi,pollution_components_co,pollution_components_no,pollution_components_no2,pollution_components_o3,pollution_components_so2,pollution_components_pm2_5,pollution_components_pm10,pollution_components_nh3
city,,,,,,,,,,,,,,,,,,,,,
cairo,stations,10000,2025-08-21 00:20:17,10800,7922173,Al ‘Atabah,200,31.2357,30.0444,299.48,...,2025-08-21 00:20:17,2,125.13,0.00,2.35,95.45,7.64,15.13,43.57,2.20
warsaw,stations,10000,2025-08-21 00:18:25,7200,756135,Warsaw,200,21.0122,52.2297,286.54,...,2025-08-21 00:20:18,1,143.78,0.00,4.29,59.71,1.51,2.48,3.66,2.86
beijing,stations,10000,2025-08-21 00:20:18,28800,1816670,Beijing,200,116.3913,39.9062,299.09,...,2025-08-21 00:20:19,5,434.70,2.14,18.55,4.46,11.23,84.24,89.73,13.21


In [4]:
#Checking dups
print(df_merged.duplicated().sum())

#Checking NA
print(df_merged.isna().any())

#Since wind gust has high probability of being null and doesn't contribute
#much in the analysis we can drop it
df_merged.dropna(axis=1, inplace=True)
print(df_merged.isna().any())


0
base                          False
visibility                    False
dt                            False
timezone                      False
id                            False
name                          False
cod                           False
coord_lon                     False
coord_lat                     False
main_temp                     False
main_feels_like               False
main_temp_min                 False
main_temp_max                 False
main_pressure                 False
main_humidity                 False
main_sea_level                False
main_grnd_level               False
wind_speed                    False
wind_deg                      False
clouds_all                    False
sys_type                      False
sys_id                        False
sys_country                   False
sys_sunrise                   False
sys_sunset                    False
weather_id                    False
weather_main                  False
weather_description       

In [5]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, cairo to beijing
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   base                        3 non-null      object        
 1   visibility                  3 non-null      int64         
 2   dt                          3 non-null      datetime64[ns]
 3   timezone                    3 non-null      int64         
 4   id                          3 non-null      int64         
 5   name                        3 non-null      object        
 6   cod                         3 non-null      int64         
 7   coord_lon                   3 non-null      float64       
 8   coord_lat                   3 non-null      float64       
 9   main_temp                   3 non-null      float64       
 10  main_feels_like             3 non-null      float64       
 11  main_temp_min               3 non-null      float64      

In [6]:
date_dim = pd.concat([
    df_merged['pollution_dt'],
    df_merged['dt'],
    df_merged['sys_sunrise'],
    df_merged['sys_sunset']
], ignore_index=True)
date_dim = pd.to_datetime(date_dim.dropna().unique())
date_dim = pd.DataFrame({'date_time': date_dim})
date_dim['date_time'] = date_dim['date_time'].dt.floor('H')
date_dim['date'] = date_dim['date_time'].dt.date
date_dim['year'] = date_dim['date_time'].dt.year
date_dim['month'] = date_dim['date_time'].dt.month
date_dim['day'] = date_dim['date_time'].dt.day
date_dim['weekday'] = date_dim['date_time'].dt.day_name()
date_dim['hour'] = date_dim['date_time'].dt.hour
date_dim['is_weekend'] = date_dim['weekday'].isin(['Saturday', 'Sunday'])
date_dim = date_dim.drop_duplicates().reset_index(drop=True)
date_dim['date_sk'] = date_dim.index
cols = ['date_sk']+[c for c in date_dim.columns if c!= 'date_sk']
date_dim = date_dim[cols]
date_dim.to_csv('date_dim.csv', index=False)

In [7]:
weather_dim = df_merged[['weather_id', 'weather_main', 'weather_description', 'weather_icon']].drop_duplicates().reset_index(drop=True)
weather_dim['weather_sk']= weather_dim.index

cols = ['weather_sk']+[c for c in weather_dim.columns if c!= 'weather_sk']
weather_dim = weather_dim[cols]
weather_dim.to_csv('weather_dim.csv', index=False)

weather_dim

,weather_sk,weather_id,weather_main,weather_description,weather_icon
0,0,800,Clear,clear sky,01n
1,1,501,Rain,moderate rain,10d


In [8]:
location_dim = df_merged.reset_index()[['city', 'sys_country', 'coord_lon', 'coord_lat', 'timezone','base']].drop_duplicates().reset_index(drop=True)
location_dim['location_sk']=location_dim.index


cols = ['location_sk']+[c for c in location_dim.columns if c not in ['location_sk']]
location_dim = location_dim[cols]
location_dim.to_csv('location_dim.csv', index=False)
location_dim

,location_sk,city,sys_country,coord_lon,coord_lat,timezone,base
0,0,cairo,EG,31.2357,30.0444,10800,stations
1,1,warsaw,PL,21.0122,52.2297,7200,stations
2,2,beijing,CN,116.3913,39.9062,28800,stations


In [9]:
pollutants_fact = pd.concat([
    df_merged.filter(regex='^pollution'),
    df_merged[['weather_id']]
    ], axis = 1).drop_duplicates().reset_index()
pollutants_fact['pollutants_sk'] = pollutants_fact.index


pollutants_fact['pollution_dt'] = pollutants_fact['pollution_dt'].dt.floor('H')

pollutants_fact = pollutants_fact.merge(
    date_dim[['date_sk', 'date_time']],
    left_on= 'pollution_dt',
    right_on='date_time',
    how='left'
).merge(
    location_dim[['city', 'location_sk']],
    on='city',
    how='left',
).merge(
    weather_dim[['weather_id', 'weather_sk']],
    on='weather_id',
    how='left'
).drop(columns=['pollution_dt', 'weather_id', 'date_time', 'city'])


cols = ['pollutants_sk', 'location_sk', 'weather_sk','date_sk']+[c for c in pollutants_fact.columns if c not in ['pollutants_sk', 'date_sk', 'location_sk', 'weather_sk']]
pollutants_fact = pollutants_fact[cols]
pollutants_fact.rename(columns=lambda col: col.replace('pollution_', '') if col.startswith('pollution_') else col, inplace=True)
pollutants_fact.to_csv('pollutants_fact.csv', index=False)
pollutants_fact

,pollutants_sk,location_sk,weather_sk,date_sk,main_aqi,components_co,components_no,components_no2,components_o3,components_so2,components_pm2_5,components_pm10,components_nh3
0,0,0,0,0,2,125.13,0.00,2.35,95.45,7.64,15.13,43.57,2.20
1,1,1,0,0,1,143.78,0.00,4.29,59.71,1.51,2.48,3.66,2.86
2,2,2,1,0,5,434.70,2.14,18.55,4.46,11.23,84.24,89.73,13.21


In [10]:
weather_fact = pd.concat([
    df_merged.filter(regex='^main'),
    df_merged[['sys_sunrise', 'sys_sunset', 'visibility', 'dt', 'weather_id']]
], axis=1).drop_duplicates().reset_index()

weather_fact['weather_fact_sk'] = weather_fact.index

weather_fact['sys_sunrise'] = weather_fact['sys_sunrise'].dt.floor('H')
weather_fact= weather_fact.merge(
    date_dim[['date_sk', 'date_time']],
    left_on='sys_sunrise',
    right_on='date_time',
    how='left',
).drop(columns=['date_time','sys_sunrise' ])
weather_fact.rename(columns={"date_sk":'sunrise_date_sk'}, inplace=True)

weather_fact['sys_sunset'] = weather_fact['sys_sunset'].dt.floor('H')
weather_fact= weather_fact.merge(
    date_dim[['date_sk', 'date_time']],
    left_on='sys_sunset',
    right_on='date_time',
    how='left',
).drop(columns=['date_time','sys_sunset' ])
weather_fact.rename(columns={"date_sk":'sunset_date_sk'}, inplace=True)
# weather_fact['sys_sunset'] = weather_fact['sys_sunset'].dt.floor('H')

weather_fact['dt'] = weather_fact['dt'].dt.floor('H')
weather_fact= weather_fact.merge(
    date_dim[['date_sk', 'date_time']],
    left_on='dt',
    right_on='date_time',
    how='left',
).drop(columns=['date_time','dt'])

weather_fact= weather_fact.merge(
    location_dim[['city', 'location_sk']],
    on='city',
    how='left'
).drop(columns=['city'])

weather_fact= weather_fact.merge(
    weather_dim[['weather_id', 'weather_sk']],
    on='weather_id',
    how='left'
).drop(columns=['weather_id'])

cols= ['weather_fact_sk', 'weather_sk','location_sk','date_sk' ,'sunrise_date_sk', 'sunset_date_sk'] + [c for c in weather_fact.columns if c not in ['weather_fact_sk', 'location_sk', 'sunrise_date_sk', 'sunset_date_sk', 'date_sk', 'weather_sk']]
weather_fact = weather_fact[cols]
weather_dim.rename(columns=lambda col: col.replace('main_', '') if col.startswith('main_') else col, inplace=True )
weather_fact.to_csv('weather_fact.csv', index=False)
weather_fact

,weather_fact_sk,weather_sk,location_sk,date_sk,sunrise_date_sk,sunset_date_sk,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_humidity,main_sea_level,main_grnd_level,visibility
0,0,0,0,0,1,3,299.48,299.48,298.85,299.48,1008,73,1008,1002,10000
1,1,0,1,0,1,4,286.54,285.87,284.76,287.95,1008,74,1008,999,10000
2,2,1,2,0,2,5,299.09,300.27,299.09,299.09,1006,97,1006,1001,10000
